In [1]:
import pandas as pd
import numpy as np
import warnings 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import re
from tqdm.notebook import tqdm

warnings.simplefilter('ignore')

In [2]:
def gnss_log_to_dataframes(path):
    '''Load GNSS Log'''
    print('Loading ' + path, flush = True)
    gnss_section_names = {'Raw', 'UncalAccel', 'UncalGyro', 'UncalMag', 'Fix', 'Status', 'OrientationDeg'}
    with open(path) as f_open:
        datalines = f_open.readlines()

    datas = {k: [] for k in gnss_section_names}
    gnss_map = {k: [] for k in gnss_section_names}
    for dataline in datalines:
        is_header = dataline.startswith('#')
        dataline = dataline.strip('#').strip().split(',')
        # skip over notes, version numbers, etc
        if is_header and dataline[0] in gnss_section_names:
            gnss_map[dataline[0]] = dataline[1:]
        elif not is_header:
            datas[dataline[0]].append(dataline[1:])

    results = dict()
    for k, v in datas.items():
        results[k] = pd.DataFrame(v, columns=gnss_map[k])
    # pandas doesn't properly infer types from these lists by default
    for k, df in results.items():
        for col in df.columns:
            if col == 'CodeType':
                continue
            results[k][col] = pd.to_numeric(results[k][col])

    return results

In [3]:
# apply tips1
# _derivedのmillisSinceGpsEpochが次のepoch(rawの)を示しているので元に戻す
def apply_tips1(raw_df, derived_df):
    # Create a new column in df_raw that corresponds to derivedの['millisSinceGpsEpoch']
    raw_df['millisSinceGpsEpoch'] = np.floor((raw_df['TimeNanos'] - raw_df['FullBiasNanos']) / 1000000.0).astype(int)
        
    # Change each value in df_derived['MillisSinceGpsEpoch'] to be the prior epoch.
    raw_timestamps = raw_df['millisSinceGpsEpoch'].unique()
    derived_timestamps = derived_df['millisSinceGpsEpoch'].unique()

    # The timestamps in derived are one epoch ahead. We need to map each epoch
    # in derived to the prior one (in Raw).
    indexes = np.searchsorted(raw_timestamps, derived_timestamps)
    from_t_to_fix_derived = dict(zip(derived_timestamps, raw_timestamps[indexes-1]))
    derived_df['millisSinceGpsEpoch'] = np.array(list(map(lambda v: from_t_to_fix_derived[v], derived_df['millisSinceGpsEpoch'])))
    return derived_df

In [4]:
# apply tips2
# Compute signal_type in df_raw.
# Map from constellation id to frequencies and signals.
CONSTEL_FREQ_TABLE = {
    0: {'UNKNOWN': (0, 999999999999)},
    1: {
        'GPS_L1': (1563000000, 1587000000),
        'GPS_L2': (1215000000, 1240000000),
        'GPS_L5': (1164000000, 1189000000)
    },
    3: {
        'GLO_G1': (1593000000, 1610000000),
        'GLO_G2': (1237000000, 1254000000)
    },
    4: {
        'QZS_J1': (1563000000, 1587000000),
        'QZS_J2': (1215000000, 1240000000),
        'QZS_J5': (1164000000, 1189000000)
    },
    5: {
        'BDS_B1C': (1569000000, 1583000000),
        'BDS_B1I': (1553000000, 1568990000),
        'BDS_B2A': (1164000000, 1189000000),
        'BDS_B2B': (1189000000, 1225000000)
    },
    6: {
        'GAL_E1': (1559000000, 1591000000),
        'GAL_E5A': (1164000000, 1189000000),
        'GAL_E5B': (1189000000, 1218000000),
        'GAL_E6': (1258000000, 1300000000)
    },
    7: {
        'IRN_S': (2472000000, 2512000000),
        'IRN_L5': (1164000000, 1189000000)
    },
}

def SignalTypeFromConstellationAndFequency(constel, freq_hz):
  'Returns the signal type as a string for the given constellation and frequency.'
  freqs = CONSTEL_FREQ_TABLE.get(constel, {})
  for id_freq_range in freqs.items():
    rng = id_freq_range[1]
    if rng[0] <= freq_hz <= rng[1]:
      return id_freq_range[0]
  return 'UNKNOWN'

# gnsslogとderivedのsvidを一致させる
def apply_tips2(raw_df):
    # Fix QZS Svids issue. 

    # The SVID of any QZS sat in derived may be changed. Since it may be a many to one relationship, we'll need to adjust the values in Raw.
    new_to_old = {1:(183, 193), 2:(184, 194, 196), 3:(187, 189, 197, 199), 4:(185, 195, 200)}
    # Maps original svid to new svid for only ConstellationType=4.
    old_to_new={}
    for new_svid, old_svids in new_to_old.items():
        for s in old_svids:
            old_to_new[s] = new_svid
        raw_df['Svid'] = raw_df.apply(lambda r: old_to_new.get(r.Svid, r.Svid) if r.ConstellationType == 4 else r.Svid, axis=1)
    return raw_df

# derivedにはCarrierFrequencyHzがないのでsignaltypeを追加
def apply_tips3(raw_df):

    signal_types = itertools.chain(*[c.keys() for c in CONSTEL_FREQ_TABLE.values()])
    sig_type_cat = pd.api.types.CategoricalDtype(categories=signal_types)
    raw_df['SignalType'] = raw_df.apply(lambda r: SignalTypeFromConstellationAndFequency(r.ConstellationType, r.CarrierFrequencyHz), axis=1).astype(sig_type_cat)
    return raw_df

In [5]:
# apply tips5
# derivedの重複している行を削除
def apply_tips5(derived_df):
    delta_millis = derived_df['millisSinceGpsEpoch'] - derived_df['receivedSvTimeInGpsNanos'] / 1e6
    where_good_signals = (delta_millis > 0) & (delta_millis < 300)
    return derived_df[where_good_signals]

In [6]:
def decode_osr(df):
    print("msg_type:", df["msg_type"].unique())
    # MSG_OBS
    # obs = df[df["msg_type"]==74].dropna(axis=1)
    # header = pd.json_normalize(obs["header"], sep="_")
    # obs = pd.concat([obs.drop(["header"], axis=1).reset_index(drop=True), header], axis=1)

    # MSG_BASE_POS_ECEF
    # 基地局のECEF座標
    # base_pos = df[df["msg_type"]==72].dropna(axis=1)

    # MSG_EPHEMERIS_GPS
    # GPS衛星のエフェメリスデータ
    gps = df[df["msg_type"]==138].dropna(axis=1)
    if len(gps) > 0:
        common = pd.json_normalize(gps["common"], sep="_")
        toc = pd.json_normalize(gps["toc"], sep="_")
        gps = pd.concat([gps.drop(["common"], axis=1).reset_index(drop=True), common], axis=1)
        gps = pd.concat([gps.drop(["toc"], axis=1).reset_index(drop=True), toc], axis=1)
        gps = gps.drop_duplicates().reset_index(drop=True)
    else:
        gps = None

    # MSG_EPHEMERIS_BDS
    bds = df[df["msg_type"]==137].dropna(axis=1)
    if len(bds) > 0:
        common = pd.json_normalize(bds["common"], sep="_")
        toc = pd.json_normalize(bds["toc"], sep="_")
        bds = pd.concat([bds.drop(["common"], axis=1).reset_index(drop=True), common], axis=1)
        bds = pd.concat([bds.drop(["toc"], axis=1).reset_index(drop=True), toc], axis=1)
        bds = bds.drop_duplicates().reset_index(drop=True)
    else:
        bds = None

    # MSG_EPHEMERIS_GAL
    gal = df[df["msg_type"]==141].dropna(axis=1)
    if len(gal) > 0:
        common = pd.json_normalize(gal["common"], sep="_")
        toc = pd.json_normalize(gal["toc"], sep="_")
        gal = pd.concat([gal.drop(["common"], axis=1).reset_index(drop=True), common], axis=1)
        gal = pd.concat([gal.drop(["toc"], axis=1).reset_index(drop=True), toc], axis=1)
        gal = gal.drop_duplicates().reset_index(drop=True)
    else:
        gal = None

    # MSG_EPHEMERIS_GLO
    glo = df[df["msg_type"]==139].dropna(axis=1)
    if len(glo) > 0:
        common = pd.json_normalize(glo["common"], sep="_")
        glo = pd.concat([glo.drop(["common"], axis=1).reset_index(drop=True), common], axis=1)
        glo["xpos"] = glo["pos"].apply(lambda x: x[0])
        glo["ypos"] = glo["pos"].apply(lambda x: x[1])
        glo["zpos"] = glo["pos"].apply(lambda x: x[2])
        glo["xvel"] = glo["vel"].apply(lambda x: x[0])
        glo["yvel"] = glo["vel"].apply(lambda x: x[1])
        glo["zvel"] = glo["vel"].apply(lambda x: x[2])
        glo["xacc"] = glo["acc"].apply(lambda x: x[0])
        glo["yacc"] = glo["acc"].apply(lambda x: x[1])
        glo["zacc"] = glo["acc"].apply(lambda x: x[2])
        glo.drop(["pos", "vel", "acc"], axis=1, inplace=True)
        glo = glo.drop_duplicates().reset_index(drop=True)
    else:
        glo = None
    # MSG_OSR
    # osr = df[df["msg_type"]==1600].dropna(axis=1)

    return gps, bds, gal, glo

In [7]:
from math import *
# import kepler

def newton_kepler(meanE, e):
    E = 2 * meanE
    eps = 1e-10
    while True:
        E -= (meanE - E + e * np.sin(E)) / (e * np.cos(E) - 1)
        if abs(meanE - E + e * np.sin(E)) < eps:
            return E

def calc_sat_position(e, t, transmission_t):
    """
    t:送信時のGPSシステムの時刻(GPS時刻)をトランジットタイム（距離／光速）で補正したもの。
    tk:時刻tとエポックタイムtoeの間の実際の合計時間差であり、週の初めまたは終わりの交差を考慮しなければならない。
    tk = t - toe
    if tk > 302,400:
        tk - 604,800
    elif tk < -302,400:
        tk + 604,800
    """
    # (1)平均角速度nの計算
    mu = 3.986005*10**14 # (m3/s2) WGS 84 value of the earth's gravitational constant forGPS user
    A = e.sqrta**2  # Semi-major axis
    n0 = sqrt((mu/(A**3))) # Computed mean motion (rad/sec)
    n = n0 + e.dn  # Mean anomaly

    # (2)平均近点離角Mの計算
    # エフェメリスの基準時刻teにおける平均近点離角M0を観測時tにおける平均近点離角Mに変換
    # tは衛星から信号が送信された時刻
    # toeは基準時刻
    toe = e.toe_tow + (e.toe_wn * 3600*24*7)
    tk = (t - toe) - transmission_t  # Time from ephemeris reference epoch

    if tk > 302400:
        tk -= 604800
    elif tk < -302400:
        tk += 604800

    M = e.m0 + n*tk # Mean anomaly

    # (3)Mean anomaly Eの計算
    # 解きたい方程式 Mk = Ek - sample.ecc*sin(Ek)
    # E = kepler.solve(M, e.ecc)
    # E, cos_true_anomaly, sin_true_anomaly = kepler.kepler(M, e.ecc)
    E = newton_kepler(M, e.ecc)

    # (4) True Anomaly vの計算
    v = atan((sqrt(1-e.ecc**2) * sin(E)) / (cos(E) - e.ecc))

    # (5)摂動補正計算
    fai = v + e.w # Argument of Latitude

    # Second Harmonic Perturbations
    u = fai + e.c_us*sin(2*fai) + e.c_uc*cos(2*fai)  # 衛星飛行方向
    r = A*(1-(e.ecc*cos(E))) + e.c_rs*sin(2*fai) + e.c_rc*cos(2*fai)  # 動径方向
    i = e.inc + e.inc_dot*tk + e.c_is*sin(2*fai) + e.c_ic*cos(2*fai)  # 軌道面に直角な方向

    # 2次元位置
    x_ = r*cos(u)
    y_ = r*sin(u)

    # 補正された昇交点赤経
    omega_dot_e = 7.2921151467*(10**-5)  # (rad/s) WGS 84 value of the earth's rotation rate
    omega = e.omega0 + (e.omegadot - omega_dot_e)*tk - (omega_dot_e*e.toe_tow)

    # ３次元位置
    x = x_*cos(omega) - y_*cos(i)*sin(omega)
    y = x_*sin(omega) + y_*cos(i)*cos(omega)
    z = y_*sin(i)
    return x,y,z

In [8]:
from pathlib import Path
root_dir = Path('../input')
data_dir = root_dir/'google-smartphone-decimeter-challenge'
train_df = pd.read_csv(data_dir / "baseline_locations_train.csv")
test_df = pd.read_csv(data_dir / "baseline_locations_test.csv")

In [9]:
output_dir = '../input/derived_v2/'
os.makedirs(output_dir, exist_ok=True)

In [10]:
const_type = pd.read_csv(data_dir / "metadata/constellation_type_mapping.csv")
const_type

,constellationType,constellationName
0,0,UNKNOWN
1,1,GPS
2,2,SBAS
3,3,GLONASS
4,4,QZSS
5,5,BEIDOU
6,6,GALILEO
7,7,IRNSS


In [18]:
# GPS:1 / GLONASS:3 / BEIDOU:5 / GALILEO:6
def satpos_estimate(derived_df, ephemeris, ctype):
    # print(f"constellationType: {ctype}")
    count = 0
    target_derived_df = derived_df[derived_df['constellationType'] == ctype]
    target_derived_df['correctedPrM'] = target_derived_df.apply(lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,axis=1)
    output_df = pd.DataFrame()

    # 衛星ID, 時刻ごとに衛星位置を求める
    gr = target_derived_df.groupby(["millisSinceGpsEpoch", "svid", "signalType"])
    for (epoch, svid, stype), df in gr:
        if len(df) != 1:
            print(epoch, svid, stype)
            print(df)
        assert len(df) == 1
        df = df.iloc[0]  

        # 観測時刻(単位:s)
        t = epoch / 1000  # 衛星位置を知りたい時刻(s)
        
        # 受信->送信に補正する用
        # 時刻が受信時のものであるためそれを補正するために擬似距離/光速で補正
        pseudorange = df['correctedPrM']
        c = 299_792_458  # 光速
        transmission_t = pseudorange / c
        
        if svid in ephemeris['sid_sat'].unique():
            e = ephemeris[(ephemeris['sid_sat']==svid)].drop_duplicates()

            # 時間の異なるeが複数ある(確認した範囲では2hのずれ)
            # 衛星位置を知りたい時間tとteが近いものを採用する
            te = e.toe_tow + (e.toe_wn * 3600*24*7)
            idx = np.argmin(np.abs(te.values - t))
            e = e.iloc[idx]
            x,y,z = calc_sat_position(e, t, transmission_t)   
            df['x'] = x
            df['y'] = y
            df['z'] = z

            # データが反転しているものがなぜかあるのでそれを補正
            if np.abs(df["x"] - df["xSatPosM"]) > 1000:
                df['x'] = -x
                df['y'] = -y
                df['z'] = -z

                # 補正してもなお大きな差がある場合はbaselineの値を使う
                if np.abs(df["x"] - df["xSatPosM"]) > 1000:
                    df['x'] = df['xSatPosM']
                    df['y'] = df['ySatPosM']
                    df['z'] = df['zSatPosM']
                    count += 1
                
        # 対象の衛星がエフェメリスデータに含まれていない場合
        else:
            df['x'] = df['xSatPosM']
            df['y'] = df['ySatPosM']
            df['z'] = df['zSatPosM']
            count += 1

        output_df = pd.concat([output_df, pd.DataFrame(df).T])
    print(f"baselineで置換した割合:{count/len(gr)}")
    return output_df.reset_index(drop=True)

In [19]:
def apply_satpos_estimate_train(args):
    (cname,pname), _ = args

    print(f"\n{cname}_{pname}")
    derived_df = pd.read_csv(data_dir/"train"/cname/pname/f'{pname}_derived.csv')
    derived_df = derived_df.drop_duplicates()

    # 時間短縮のためすでにファイルがあるのであれば省略
    # if os.path.exists(output_dir + f'{cname}_{pname}_derived.csv'):
    #     return derived_df

    # pseudorangeの修正
    derived_df['correctedPrM'] = derived_df.apply(lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,axis=1)
    gnss_df = gnss_log_to_dataframes(str(data_dir / "train" / cname / pname / f'{pname}_GnssLog.txt'))
    raw_df = gnss_df['Raw']
    derived_df = apply_tips1(raw_df, derived_df)
    derived_df = apply_tips5(derived_df)


    target_date = int(re.sub('-', '', cname)[:8])

    if os.path.exists(f"../google-sdc-corrections/osr/json/{cname}.json"):
        osr_df = pd.read_json(f"../google-sdc-corrections/osr/json/{cname}.json", lines=True)
    else:
        json_list = glob.glob("../google-sdc-corrections/osr/json/*.json")
        json_list = [p for p in json_list if len(p.split('-'))==4]
        osr_df = pd.DataFrame()
        for file_path in json_list:
            file_name = file_path.split('/')[-1] 
            start_date = int(file_name[:8])
            end_date = int(file_name[13:21])

            # 1こ以上ある場合に対処      
            if (start_date <= target_date)&(target_date<=end_date):
                _osr_df = pd.read_json(file_path, lines=True)
                osr_df = pd.concat([osr_df, _osr_df]).reset_index(drop=True)

        if len(osr_df) == 0:
            print(f"There are not osr data in {cname}_{pname}")
            return derived_df

    osr_df.drop(["payload"], axis=1, inplace=True)

    gps, bds, gal, glo = decode_osr(osr_df)

    df_list = []
    for ctype, df in derived_df.groupby(['constellationType']):

        if ctype == 1:
            df = satpos_estimate(df, gps, ctype)
        # if ctype == 3:
        #     df = satpos_estimate_glonass(df, glo, ctype)
        # elif ctype == 5:
        #     df = satpos_estimate(df, bds, ctype)
        elif ctype == 6:
            df = satpos_estimate(df, gal, ctype)
        else:
            df['x'] = df['xSatPosM'].to_numpy()
            df['y'] = df['ySatPosM'].to_numpy()
            df['z'] = df['zSatPosM'].to_numpy()
        
        # baselineとの差が異常に大きい場合はbaselineのものを格納
        if ((df['x'] - df['xSatPosM']).abs().max() > 3)|((df['y'] - df['ySatPosM']).abs().max() > 3):
            df['x'] = df['xSatPosM'].to_numpy()
            df['y'] = df['ySatPosM'].to_numpy()
            df['z'] = df['zSatPosM'].to_numpy()
        
        df_list.append(df)

    new_derived_df = pd.concat(df_list).reset_index(drop=True)
    print(f"大きな差がないかのチェック(x):{(new_derived_df['xSatPosM'] - new_derived_df['x']).max()}")

    new_derived_df = new_derived_df.drop(["xSatPosM", "ySatPosM", "zSatPosM"], axis=1).rename(columns={"x":"xSatPosM", "y":"ySatPosM", "z":"zSatPosM"})
    print(derived_df.shape, new_derived_df.shape)
    new_derived_df.to_csv(output_dir + f'{cname}_{pname}_derived.csv', index=False)
    return new_derived_df

In [20]:
def apply_satpos_estimate_test(args):
    (cname,pname), _ = args

    print(f"\n{cname}_{pname}")
    derived_df = pd.read_csv(data_dir/"test"/cname/pname/f'{pname}_derived.csv')
    derived_df = derived_df.drop_duplicates()

    # 時間短縮のためすでにファイルがあるのであれば省略
    # if os.path.exists(output_dir + f'{cname}_{pname}_derived.csv'):
    #     return derived_df

    # pseudorangeの修正
    derived_df['correctedPrM'] = derived_df.apply(lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,axis=1)
    gnss_df = gnss_log_to_dataframes(str(data_dir / "test" / cname / pname / f'{pname}_GnssLog.txt'))
    raw_df = gnss_df['Raw']
    derived_df = apply_tips1(raw_df, derived_df)
    derived_df = apply_tips5(derived_df)


    target_date = int(re.sub('-', '', cname)[:8])

    if os.path.exists(f"../google-sdc-corrections/osr/json/{cname}.json"):
        osr_df = pd.read_json(f"../google-sdc-corrections/osr/json/{cname}.json", lines=True)
    else:
        json_list = glob.glob("../google-sdc-corrections/osr/json/*.json")
        json_list = [p for p in json_list if len(p.split('-'))==4]
        osr_df = pd.DataFrame()
        for file_path in json_list:
            file_name = file_path.split('/')[-1] 
            start_date = int(file_name[:8])
            end_date = int(file_name[13:21])

            # 1こ以上ある場合に対処      
            if (start_date <= target_date)&(target_date<=end_date):
                _osr_df = pd.read_json(file_path, lines=True)
                osr_df = pd.concat([osr_df, _osr_df]).reset_index(drop=True)

        if len(osr_df) == 0:
            print(f"There are not osr data in {cname}_{pname}")
            return derived_df

    osr_df.drop(["payload"], axis=1, inplace=True)

    gps, bds, gal, glo = decode_osr(osr_df)

    df_list = []
    for ctype, df in derived_df.groupby(['constellationType']):

        if ctype == 1:
            df = satpos_estimate(df, gps, ctype)
        # if ctype == 3:
        #     df = satpos_estimate_glonass(df, glo, ctype)
        # elif ctype == 5:
        #     df = satpos_estimate(df, bds, ctype)
        elif ctype == 6:
            df = satpos_estimate(df, gal, ctype)
        else:
            df['x'] = df['xSatPosM'].to_numpy()
            df['y'] = df['ySatPosM'].to_numpy()
            df['z'] = df['zSatPosM'].to_numpy()
        
        # baselineとの差が異常に大きい場合はbaselineのものを格納
        if ((df['x'] - df['xSatPosM']).abs().max() > 3)|((df['y'] - df['ySatPosM']).abs().max() > 3):
            df['x'] = df['xSatPosM'].to_numpy()
            df['y'] = df['ySatPosM'].to_numpy()
            df['z'] = df['zSatPosM'].to_numpy()
        
        df_list.append(df)

    new_derived_df = pd.concat(df_list).reset_index(drop=True)
    print(f"大きな差がないかのチェック(x):{(new_derived_df['xSatPosM'] - new_derived_df['x']).max()}")

    new_derived_df = new_derived_df.drop(["xSatPosM", "ySatPosM", "zSatPosM"], axis=1).rename(columns={"x":"xSatPosM", "y":"ySatPosM", "z":"zSatPosM"})
    print(derived_df.shape, new_derived_df.shape)
    new_derived_df.to_csv(output_dir + f'{cname}_{pname}_derived.csv', index=False)
    return new_derived_df

In [21]:
import multiprocessing

gr = train_df.groupby(['collectionName','phoneName'])
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(apply_satpos_estimate_train, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
all_derived_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch']).reset_index(drop=True)     

  0%|          | 0/73 [00:00<?, ?it/s]


2020-05-14-US-MTV-1_Pixel4
2020-05-14-US-MTV-1_Pixel4XLModded

2020-05-14-US-MTV-2_Pixel4


2020-05-14-US-MTV-2_Pixel4XLModded

2020-05-21-US-MTV-1_Pixel4
2020-05-21-US-MTV-2_Pixel4

2020-05-21-US-MTV-2_Pixel4XL

2020-05-29-US-MTV-1_Pixel4

2020-05-29-US-MTV-1_Pixel4XL

2020-05-29-US-MTV-1_Pixel4XLModded

2020-05-29-US-MTV-2_Pixel4


2020-05-29-US-MTV-2_Pixel4XL
2020-06-04-US-MTV-1_Pixel4

2020-06-04-US-MTV-1_Pixel4XL
2020-06-04-US-MTV-1_Pixel4XLModded


2020-06-05-US-MTV-1_Pixel4

2020-06-05-US-MTV-1_Pixel4XL

2020-06-05-US-MTV-1_Pixel4XLModded


2020-06-05-US-MTV-2_Pixel4
2020-06-05-US-MTV-2_Pixel4XL

2020-06-11-US-MTV-1_Pixel4

2020-06-11-US-MTV-1_Pixel4XL


2020-07-08-US-MTV-1_Pixel4XL
2020-07-08-US-MTV-1_Pixel4

Loading ../input/google-smartphone-decimeter-challenge/train/2020-05-14-US-MTV-2/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/train/2020-06-05-US-MTV-1/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-sm

In [17]:
import multiprocessing

gr = test_df.groupby(['collectionName','phoneName'])
processes = multiprocessing.cpu_count()
with multiprocessing.Pool(processes=processes) as pool:
    dfs = pool.imap_unordered(apply_satpos_estimate_test, gr)
    dfs = tqdm(dfs, total=len(gr))
    dfs = list(dfs)
all_derived_df = pd.concat(dfs).sort_values(['collectionName', 'phoneName', 'millisSinceGpsEpoch']).reset_index(drop=True)     


2020-05-15-US-MTV-1_Pixel4


  0%|          | 0/48 [00:00<?, ?it/s]


2020-05-15-US-MTV-1_Pixel4XL
2020-05-28-US-MTV-1_Pixel4XL
2020-05-28-US-MTV-2_Pixel4

2020-05-28-US-MTV-1_Pixel4



2020-05-28-US-MTV-2_Pixel4XL
2020-05-28-US-MTV-2_Pixel4XLModded


2020-06-04-US-MTV-2_Pixel4
2020-06-04-US-MTV-2_Pixel4XL


2020-06-04-US-MTV-2_Pixel4XLModded
2020-06-10-US-MTV-1_Pixel4
2020-06-10-US-MTV-1_Pixel4XL

2020-06-10-US-MTV-1_Pixel4XLModded



2020-06-10-US-MTV-2_Pixel4

2020-06-10-US-MTV-2_Pixel4XL

2020-06-10-US-MTV-2_Pixel4XLModded
2020-08-03-US-MTV-2_Mi8


2020-08-03-US-MTV-2_Pixel4
2020-08-03-US-MTV-2_Pixel4XL
2020-08-13-US-MTV-1_Mi8
2020-08-13-US-MTV-1_Pixel4
2021-03-16-US-MTV-2_Pixel4Modded





2021-03-16-US-MTV-2_SamsungS20Ultra
2021-03-16-US-RWC-2_Pixel4XL

Loading ../input/google-smartphone-decimeter-challenge/test/2020-05-28-US-MTV-2/Pixel4XLModded/Pixel4XLModded_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/test/2021-03-16-US-RWC-2/Pixel4XL/Pixel4XL_GnssLog.txt
Loading ../input/google-smartphone-decimeter-challenge/test/2021-03